<a href="https://colab.research.google.com/github/iyioladaniel/Machine_Learning/blob/master/KFold%2C_Cross_Val%2C_RFC_CV%2C_DTC_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

file_url='https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter07/Dataset/car.data'

# data doesn't have headers, so let's create headers
_headers = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'car']
df=pd.read_csv(file_url, names=_headers, index_col=None)
df.info()

training_df, eval_df=train_test_split(df,train_size=0.8, random_state=0)
training_df.head()
eval_df.head()

In [0]:
#K-Fold
from sklearn.model_selection import KFold
n_splits=5
_kf=KFold(n_splits=n_splits)
#splits data into 5 sets.

indices=_kf.split(df)
type(indices)
#indices is a generator

#first set
train_indices, val_indices=next(indices)

train_df=df.drop(val_indices)
val_df=df.drop(train_indices)
#print(train_df.info(),val_df.info())

_t, _v=[],[]
_indices=_kf.split(df)

#iterate over indices generator and create DF
for i in range(n_splits):
  train_idx, val_idx=next(_indices)
  _train_df=df.drop(val_idx)
  _t.append(_train_df)
  _val_df=df.drop(train_idx)
  _v.append(_val_df)

#Iterate over training list
for d in _t:
  print(d.info())

#Iterate over validation list
for d in _v:
  print(d.info())

In [65]:
_df=pd.get_dummies(df, columns=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])
_df.head()

features=_df.drop(['car'], axis=1).values
labels=_df[['car']].values

from sklearn.linear_model import LogisticRegression
log_model=LogisticRegression()

from sklearn.model_selection import cross_val_score
_scores=cross_val_score(log_model, features, labels.ravel(), cv=5)
print('R^2 for 5 sets:{}'.format(_scores))

R^2 for 5 sets:[0.70231214 0.84971098 0.74566474 0.75652174 0.76231884]


In [67]:
#using LogRegCV
from sklearn.linear_model import LogisticRegressionCV
model=LogisticRegressionCV(max_iter=2000, multi_class='auto', cv=5)

model.fit(features, labels.ravel())
#df.ravel() to convert to 1D array or vector
r2=model.score(features,labels.ravel())
print('R2 Score:{}'.format(r2))

R2 Score:0.9456018518518519


In [74]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

dtc=DecisionTreeClassifier()
params={'max_depth':np.arange(1,8)}
dtc_cv=GridSearchCV(dtc, param_grid=params, cv=5)
dtc_cv.fit(features, labels)

print('Tuned Decision Tree Parameters:{}'.format(dtc_cv.best_params_))
print('Best Score:{}'.format(dtc_cv.best_score_))

#to access best model
model=dtc_cv.best_estimator_
model

Tuned Decision Tree Parameters:{'max_depth': 2}
Best Score:0.7778822149618833


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=2, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [78]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

rfc=RandomForestClassifier()
params={'n_estimators':[500,1000,2000], 'max_depth':np.arange(1,8)}
rfc_cv=RandomizedSearchCV(rfc, param_distributions=params, cv=5)
rfc_cv.fit(features, labels.ravel())

print("Tuned Random Forest Parameters: {}".format(rfc_cv.best_params_))
print('Best Score:{}'.format(rfc_cv.best_score_))

model=rfc_cv.best_estimator_
model

Tuned Random Forest Parameters: {'n_estimators': 1000, 'max_depth': 5}
Best Score:0.762211610957527


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)